#Preparation


## Import and Mount Drive



In [1]:

import pandas as pd
import json 
import csv
from pytz import all_timezones
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


## Load JSON Data

In [2]:
READ_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/dataframes'
WRITE_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /Neo4j/triples'
def read_json(filename, path):
  
  with open(path + '/'+ filename, 'r') as f: 
    read = pd.read_json(f, orient='records', keep_default_dates=False, precise_float=True) 
  return pd.DataFrame(read)

def write_data(filename, data):

  with open(WRITE_PATH + '/' + filename, 'w') as f:
    write = csv.writer(f)
    for row in data:
      write.writerow(row)

def write_dict(filename, dct, path):
  
  with open(path + '/' + filename, 'w') as f:
    writer = csv.writer(f)
    for k,v in dct.items():
      writer.writerow([k,v])

def read_csv(filename, path):

  lst = []
  with open(path + '/' + filename, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      lst.append(row)
  return lst

def read_data(filename):
  d = {}
  with open(filename, 'r') as f:
    reader = csv.reader(f)
    for k, v in reader:
      d[k] = v
  return v



# Import Dataframes

### Read Document Data

In [3]:

pd.set_option('precision', 30)
tweet_df = read_json('tweet_data.json', READ_PATH)
tweet_df = tweet_df.where(pd.notnull(tweet_df), None)
tweet_df['timestamp'] = pd.to_datetime(tweet_df['timestamp'], format='%a %b %d %H:%M:%S +0000 %Y')
tweet_df['timestamp'] = tweet_df['timestamp'].dt.tz_localize('US/Pacific', ambiguous='NaT', nonexistent='shift_forward')
tweet_df['timestamp'] = tweet_df['timestamp'].dt.tz_localize(None, ambiguous='NaT', nonexistent='shift_forward')
tweet_df['timestamp-str'] = tweet_df['timestamp'].dt.strftime('%d-%b-%y')
tweet_dict = tweet_df.to_dict(orient='records')

In [ ]:
display(tweet_df)

,tweet-id,user-id,user-screen_name,verified,text,cleaned_text,hashtags,mentions,in-reply-to,quoted,timestamp,followers-count,replies_count,quote_count,replies,quotes,nmf_keywords,nmf_sentiment_dict,nmf_topics,nmf_topic_distribution,sentiment_score,keywords,timestamp-str
0,1237634687520014336,1289202337,samstahmandy,False,Realized my roommate was patient 0 for coronav...,"[realize, roommate, patient, coronavirus, know...",{},{},None,None,2020-03-11 07:01:18,1042,0,0,[],[],[],{},"[0, 1, 7, 10, 15, 21, 53, 76]","[[0, 0.004532880321274729], [1, 0.000207206580...",0,"{'0': ['patient', '1.0'], '1': ['wanna', '0.81...",11-Mar-20
1,1237635197228560384,264516454,eddiejaylee,False,@aydild7 if they don’t keep you in quarantine ...,"[keep, quarantine, first]",{},{},1237633071186400000,None,2020-03-11 07:03:19,989,0,0,[],[],[],{},"[0, 1, 57, 86]","[[0, 0.0], [1, 0.1807539134281934], [57, 0.431...",0,"{'0': ['damn', '0.68732256'], '1': ['quarantin...",11-Mar-20
2,1237635597734313984,260075530,bebephoquechic,False,I wonder if Disney will nix the “no cancellati...,"[wonder, disney, cancellation”, contract, term...",{},{},None,None,2020-03-11 07:04:55,35,0,0,[],[],[],{},"[0, 1, 7, 38, 48, 67, 76, 84, 88, 93]","[[0, 0.6446746210187521], [1, 0.0], [7, 0.0241...",0,"{'0': ['covid', '1.0'], '1': ['wanna', '0.7197...",11-Mar-20
3,1237635660397215744,855131340,_kimru,False,CORONAVIRUS LOOPHOLE!!!!! If you don’t have in...,"[coronavirus, loophole, insurance, can't, affo...",{},{},None,1237117540297570048,2020-03-11 07:05:10,432,0,0,[],[],[],{},"[0, 1, 7, 8, 20, 32, 34, 53, 64, 68, 85]","[[0, 0.0], [1, 0.0], [7, 0.05051254265213723],...",0.644900000000000028776980798284,"{'0': ['damn', '0.7636737'], '1': ['bore', '0....",11-Mar-20
4,1237635676813684736,4879886542,superheroejay,False,So I saw a friend post the Corona Seltzer with...,"[friend, post, corona, seltzer, girl, beach, l...",{},{},None,None,2020-03-11 07:05:14,319,0,0,[],[],[],{},"[0, 1, 4, 48, 64]","[[0, 0.0], [1, 0.0005518106976458455], [4, 0.4...",0.539000000000000034638958368305,"{'0': ['post', '1.0'], '1': ['post', '1.0'], '...",11-Mar-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613944,1405414881105379328,1174744347406782464,TooYurntBrian,False,I been working out every day and today was my ...,"[work, every, today, first, rest, feel, failur...",{},{},None,None,2021-06-17 06:39:53,440,0,0,[],[],[],{},"[0, 1, 6, 16, 21, 36, 47, 79, 86]","[[0, 0.0], [1, 0.0], [6, 0.10277289219990748],...",-0.855500000000000038191672047105,"{'0': ['cause', '0.7878567'], '1': ['wanna', '...",17-Jun-21
613945,1405414978450984960,113728767,jasonlytle,True,when i walk into a store in the usa and a perk...,"[walk, store, perky, employee, wear, mask, vel...",{},{},None,None,2021-06-17 06:40:16,8121,0,0,[],[],[],{},"[0, 1, 2, 88]","[[0, 0.0], [1, 0.0], [2, 0.7766833801675525], ...",-0.594999999999999973354647408996,"{'0': ['damn', '0.7055538'], '1': ['crazy', '0...",17-Jun-21
613946,1405415911763312640,1348476612119957504,rGuida79,False,@SBF_Alameda Agreed almost on allYou are smart...,"[agree, almost, allyou, smartbut, wrong, covid...",{},{},1405401912862670080,None,2021-06-17 06:43:59,118,0,0,[],[],[would],{'would': -0.4939},"[0, 1, 10, 19, 38, 44, 52, 83]","[[0, 0.042085011815152173], [1, 0.0], [10, 0.0...",-0.859099999999999974775732880516,"{'0': ['covid', '1.0'], '1': ['wanna', '0.7587...",17-Jun-21
613947,1405416089073315840,260821810,nursealexiss,False,If #JHACO inspections were “unnecessary” durin...,"[inspection, unnecessary”, pandemic, year, unn...",{},{},None,None,2021-06-17 06:44:41,9125,0,0,[],[],[],{},"[0, 1, 3, 25, 33, 39, 41, 76, 88]","[[0, 0.0005811509518193407], [1, 0.0], [3, 0.1...",-0.0773999999999999965805130841545,"{'0': ['hospital', '1.0'], '1': ['bore', '0.87...",17-Jun-21


### Read User Data

In [4]:

pd.set_option('precision', 30)
user_df = read_json('user_data.json', '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/dataframes')
user_df = user_df.where(pd.notnull(user_df), None)
user_dict = user_df.to_dict(orient = 'records')
print(user_df.shape)

(91479, 13)


### Read Topic Data

In [5]:

pd.set_option('precision', 30)
topic_df = read_json('topic_data_with_peaks_new.json', '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/dataframes')
topic_df = topic_df.where(pd.notnull(topic_df), None)
topic_dict = topic_df.to_dict(orient = 'records')
print(topic_df.shape)

(100, 7)


### Read Event Data


In [6]:

event_data = read_csv('event_data.csv', '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/dataframes')
event_list = []
event_dict = {}
# manually format first dates
event_list.append([event_data[1][0], event_data[1][1]])
event_dict[0] = event_data[1][0]
event_list.append([event_data[2][0], event_data[2][1].split('.')[0] + "."])
event_dict[1] = event_data[2][0] 
event_list.append(['04-Mar-20', event_data[2][1].split('.')[1][15:] + "."])
event_dict[2] = '04-Mar-20'
j = 3
for i in event_data[3:]:
  event_list.append([i[0], i[1]])
  event_dict[j] = i[0]  
  j+= 1
# event_df = pd.DataFrame(event_list, columns=['date', 'policy'])
print(len(event_list))

366


In [ ]:
id2event = {v+1000:k[1] for v,k in enumerate(event_list)}
write_dict('id2event.csv', id2event, WRITE_PATH)

# Pandas Attempt (abandoned)

In [ ]:
hashtags = set()
users = set(user_df['user-screen_name'].to_list())
dates = set()
for tweet in tweet_dict:
  if tweet['hashtags']:
    for tag in tweet['hashtags']:
      hashtags.add(tag)
  if tweet['mentions']: 
    for men in tweet['mentions']:
      users.add(men)
  dates.add(tweet['timestamp-str'])


In [ ]:
tweet_ent = pd.DataFrame(tweet_df, columns=['tweet-id'])
tweet_ent = tweet_ent.rename(columns={'tweet-id':'entity'})
topic_ent = pd.DataFrame(topic_df, columns=['topic_id'])
topic_ent = topic_ent.rename(columns={'topic_id':'entity'})
event_ent = pd.DataFrame(event_df, columns=['policy'])
event_ent = event_ent.rename(columns={'policy':'entity'})
hashtag_ent = pd.DataFrame(list(hashtags), columns=['hashtag'])
hashtag_ent = hashtag_ent.rename(columns={'hashtag':'entity'})
user_ent = pd.DataFrame(list(users), columns=['users'])
user_ent = user_ent.rename(columns={'users':'entity' })
date_ent = pd.DataFrame(list(dates), columns=['dates'])
date_ent = date_ent.where(pd.notnull(date_ent), None)
date_ent = date_ent.rename(columns={'dates':'entity' })

NameError: ignored

In [ ]:
display(tweet_ent)
display(topic_ent)
display(event_ent)
display(hashtag_ent)
display(user_ent)
display(date_ent)

In [ ]:
ent_dfs = [tweet_ent, topic_ent, event_ent, hashtag_ent, user_ent, date_ent]
entities = pd.concat(ent_dfs, ignore_index=True)
entities = entities.astype(str)

# Create Indices

## Create Entity Indices

In [ ]:
dates = set()
for tweet in tweet_dict:
  dates.add(str(tweet['timestamp-str']))
print(len(dates))

465


In [ ]:
write_data('dates.csv', list(dates))

In [9]:
tags = set()

In [10]:
all_entities = set()
keywords = set()
for tweet in tweet_dict:
    all_entities.add(str(tweet['tweet-id']))
    all_entities.add(tweet['timestamp-str'])
    #dates.add(tweet['timestamp-str'])
    if tweet['timestamp-str'] is None:
      print('None')

    for tag in tweet['hashtags']:
      all_entities.add(str(tag))
      tags.add(tag)
    for men in tweet['mentions']:
      all_entities.add(str(men))
  
for user in user_dict:
  all_entities.add(str(user['user-screen_name']))

for topic in topic_dict:
  all_entities.add(str(topic['topic_id']))
  #print(topic['keywords'])
  for key in topic['keywords']:
    all_entities.add(str(key))
    keywords.add(str(key))
    #print(key)
for event in event_list:
    all_entities.add(str(event[1]))


print(len(all_entities))


711783


In [11]:
print(len(tags))

2540


In [ ]:
# write_data('keywords.csv', keywords)

In [ ]:
ent_list = list(all_entities)
entity2id = {k:v for v,k in enumerate(ent_list)}
id2entity = {v:k for v,k in enumerate(ent_list)}

In [ ]:
print(len(entity2id))

711783


### Write Entity files

In [ ]:
write_dict('id2entity-peaks.csv', id2entity, WRITE_PATH)
write_dict('entity2id-peaks.csv', entity2id, WRITE_PATH)

## Create Relation Indices


Relations: 


*   replied_by: tweet -> tweet
*   quoted_by: tweet -> tweet
*   tweeted: user -> tweet
*   has_hashtag: tweet -> hashtag
*   mentioned: tweet -> user
*   created_on: tweet -> date
*   announced_on: event -> date
*   has_keyword: tweet -> keyword *
*   associated_with: keyword -> topic *
*   in_topic: tweet -> topic *
*   has_changepoint: topic -> date
*   has_peak: topic -> date












In [ ]:

relations = ['replied_by', 'quoted_by', 'tweeted', 'has_hashtag', 'mentioned', 
             'created_on', 'announced_on', 'has_keyword', 'in_topic', 'associated_with', 
             'has_changepoint'] 
          #'has_peak']
relation2id = {}
id2relation = {}
j = 0 
for r in relations:
  relation2id[r] = j
  id2relation[j] = r
  j += 1


### Write Relation Files

In [ ]:

# write_dict('relation2id-peaks.csv', relation2id, WRITE_PATH)
# write_dict('id2relation-peaks.csv', id2relation, WRITE_PATH)

# Create Triples

## Tweet-to-Tweet triples

In [ ]:

all_relations=[]
tweet_relations = []

for tweet in tweet_dict:
  
  if tweet['replies']: 
    for reply in tweet['replies']:
      if str(reply) in entity2id:
        relation = []
        relation.append(entity2id[str(tweet['tweet-id'])])
        relation.append(entity2id[str(reply)])
        relation.append(relation2id['replied_by'])
        relation.append(1.0)
        tweet_relations.append(relation)
        #print(relation)
  if tweet['quotes']:
    for quote in tweet['quotes']:
      if str(quote) in entity2id:
        relation = []
        relation.append(entity2id[str(tweet['tweet-id'])])
        relation.append(entity2id[str(quote)])
        relation.append(relation2id['quoted_by'])
        relation.append(1.0)
        tweet_relations.append(relation)
        #print(relation)
  relation = []
  relation.append(entity2id[str(tweet['tweet-id'])])
  if tweet['timestamp-str'] is None:
    print("Oops")
  relation.append(entity2id[tweet['timestamp-str']])
  relation.append(relation2id['created_on']) 
  relation.append(1.0)
  tweet_relations.append(relation)
  #print(relation)

all_relations = tweet_relations


## Event-to-Date triples


In [ ]:

event_relations = []
for i in range(len(event_list)):
  if event_dict[i] in entity2id:
    relation = []
    relation.append(entity2id[str(event_list[i][1])])
    relation.append(entity2id[str(event_dict[i])])
    relation.append(relation2id['announced_on'])
    relation.append(1.0)
    event_relations.append(relation)

all_relations = [*all_relations, *event_relations]

## User-to-Tweet Relations

In [ ]:
user_relations = []

for user in user_dict:
  if user['tweets-in-dataset']:
    for tweet in user['tweets-in-dataset']:
      relation = []
      relation.append(entity2id[str(user['user-screen_name'])])
      relation.append(entity2id[str(tweet)])
      relation.append(relation2id['tweeted']) 
      relation.append(1.0)
      user_relations.append(relation)
      #print(relation)
all_relations = [*all_relations, *user_relations]
#write_data('user_relations_20mar11-21jun16.csv', user_relations)

## Tweet-to-Hashtag Triples

In [ ]:


hashtag_relations = []

for tweet in tweet_dict:
  for tag in tweet['hashtags']:
    temp = [] 
    temp.append(entity2id[str(tweet['tweet-id'])])
    temp.append(entity2id[str(tag)])
    temp.append(relation2id['has_hashtag'])
    temp.append(1.0)
    hashtag_relations.append(temp)
    #print(temp)
all_relations = [*all_relations, *hashtag_relations]
#write_data('hashtag_relations_20mar11-21jun16.csv', hashtag_relations)

## Tweet-to-Mention Triples

In [ ]:


mention_relations = []
for tweet in tweet_dict:
  for men in tweet['mentions']:
    temp = [] 
    temp.append(entity2id[str(tweet['tweet-id'])])
    temp.append(entity2id[str(men)])
    temp.append(relation2id['mentioned'])
    temp.append(1.0)
    mention_relations.append(temp)
    #print(temp)
all_relations = [*all_relations, *mention_relations]

#write_data('mention_relations_20mar11-21jun16.csv', mention_relations)

## Keyword-to-Topic Triples 

In [ ]:
keyword_topic_relations=[]

for topic in topic_dict:
  for keyword in topic['keywords_weightings']:
    relation = []
    relation.append(entity2id[str(keyword[0])])
    relation.append(entity2id[str(topic['topic_id'])])
    relation.append(relation2id['associated_with']) 
    relation.append(float(keyword[1]))
    keyword_topic_relations.append(relation)
all_relations = [*all_relations, *keyword_topic_relations]

## Tweet-to-Topic Triples 

In [ ]:
print(len([])==0)

True


In [ ]:

topic_tweet_relations = []
for tweet in tweet_dict:
  if tweet['nmf_topic_distribution'] != 0:
    for topic in tweet['nmf_topic_distribution']:
      relation = []
      relation.append(entity2id[str(tweet['tweet-id'])])
      relation.append(entity2id[str(topic[0])])
      relation.append(relation2id['in_topic'])
      relation.append(float(topic[1]))
      topic_tweet_relations.append(relation)
  else:
    print(str(tweet['tweet-id']))
all_relations = [*all_relations, *topic_tweet_relations]


## Tweet-to-Keyword Triples


In [ ]:

tweet_keyword_relations = []
for tweet in tweet_dict:
  for k in tweet['keywords'].values():
    relation = [] 
    relation.append(entity2id[str(tweet['tweet-id'])])
    relation.append(entity2id[str(k[0])]) 
    relation.append(relation2id['has_keyword'])
    relation.append(float(k[1])) 
    tweet_keyword_relations.append(relation)

all_relations = [*all_relations, *tweet_keyword_relations]

# Topic-to-Date Relations

In [ ]:
peaks = []

In [ ]:
topic_date_relations = []
for topic in topic_dict:
  # for date in topic['change_points']:
  #   if date != '17-Jun-21':
  #     relation = []
  #     relation.append(entity2id[str(topic['topic_id'])])
  #     relation.append(entity2id[str(date)])
  #     relation.append(relation2id['has_changepoint'])
  #     relation.append(float(1.0))
  #     topic_date_relations.append(relation)
  for peak in topic['peaks']:
    peaks.append(peak) 
    relation=[]
    relation.append(entity2id[str(topic['topic_id'])])
    relation.append(entity2id[str(peak)])
    relation.append(relation2id['has_peak'])
    relation.append(float(1.0))
    topic_date_relations.append(relation)

all_relations = [*all_relations, *topic_date_relations]

In [ ]:
import collections 
d = collections.Counter(peaks)
print

<function print>

In [ ]:
import collections
d = {}
for topic in topic_dict:
  t = []
  for date in topic['peaks']:
    t.append(date)
    if date != '17-Jun-21':
      if date in d.keys():
        d[date] += 1
      else:
        d[date] = 1
  print(topic['topic_id'],t)
for k,v in d.items():
  print(k, v)

0 ['20-Mar-20', '02-Oct-20']
1 ['20-Mar-20', '02-Oct-20']
2 ['27-Jun-20', '02-Oct-20', '12-Jun-20', '20-Jul-20', '08-Apr-20', '21-May-20', '13-May-21', '28-Apr-20', '17-Nov-20', '15-Jun-21', '03-Sep-20', '16-Mar-20', '02-Mar-21', '13-Jan-21', '11-Aug-20']
3 ['17-Mar-20', '15-May-20', '22-Apr-20', '30-Jun-20']
4 ['12-Mar-20', '25-Mar-20']
5 ['20-Mar-20', '03-Apr-20', '22-Apr-20', '23-Jun-20', '28-Nov-20']
6 ['18-Dec-20', '13-Apr-21', '09-Nov-20', '12-Mar-21', '12-Jan-21', '20-Mar-20', '31-Jan-21', '14-May-21', '25-Mar-21', '03-Dec-20', '15-May-20']
7 ['12-Mar-20', '27-Mar-20', '24-Apr-20']
8 ['02-Oct-20', '12-Mar-20', '28-Oct-20']
9 ['18-Mar-20', '01-Apr-20', '23-Oct-20', '20-Jul-20', '30-Sep-20', '06-Jul-20', '21-May-20']
10 ['16-Mar-20', '29-Mar-20', '02-Oct-20', '17-Apr-20', '28-Oct-20']
11 ['16-Mar-20', '01-Apr-20', '12-Jun-20', '24-Apr-20', '27-Jun-20', '02-Oct-20']
12 ['13-Mar-20', '29-Mar-20', '26-Jun-20', '15-Jul-20', '24-Apr-20', '31-Oct-20', '16-Nov-20', '03-Oct-20', '10-Jun-2

In [ ]:
print(len(all_relations))



10103196


In [ ]:
print(max([x[0] for x in all_relations]))
print(max(id2entity.keys()))

711782
711782


# Write Relations File

In [ ]:
print(max([x[0] for x in all_relations]))
print(len(id2entity.keys()))
print(len(entity2id.keys()))

711782
711783
711783


In [ ]:
write_data('all_relations-peaks-v2.csv', all_relations) 

In [ ]:
print(len(all_relations))

10103143
